In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test  = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

print(train.shape)
print(test.shape)
train.head()


In [ ]:
import matplotlib.pyplot as plt

train["SalePrice"].describe()
np.log(train["SalePrice"]).hist(bins=50)
plt.show()


In [ ]:
y = np.log(train["SalePrice"])
train.drop("SalePrice", axis=1, inplace=True)

all_data = pd.concat([train, test], axis=0)
all_data.shape


In [ ]:
for col in all_data.columns:
    if all_data[col].dtype == "object":
        all_data[col] = all_data[col].fillna("None")
    else:
        all_data[col] = all_data[col].fillna(all_data[col].median())


In [ ]:
all_data = pd.get_dummies(all_data)
all_data.shape


In [ ]:
X_train = all_data[:len(y)]
X_test  = all_data[len(y):]


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score

model = Lasso(alpha=0.0005, random_state=42)

rmse = -cross_val_score(
    model, X_train, y,
    scoring="neg_root_mean_squared_error",
    cv=5
).mean()

print("CV RMSE:", rmse)


In [ ]:
model.fit(X_train, y)
pred = np.exp(model.predict(X_test))


In [ ]:
import os
os.listdir('/kaggle/working')


In [ ]:
submission = pd.DataFrame({
    "Id": test["Id"],
    "SalePrice": pred
})

submission.to_csv("submission.csv", index=False)
submission.head()
